# Import Packages
Lets load all the needed packages for this notebook:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# The Dataset
For this notebook we will use the Tabular Playground Series - Feb 2022 competition dataset.

Let's define the path to the dataset:

In [ ]:
data: pd.DataFrame = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')
data.info()

# Quick Look at the Data
Let’s take a look at the top five rows:

In [ ]:
data.head()

# Remove any columns that aren't needed from the dataset.

In [ ]:
data = data.drop(['row_id'],axis=1)
data.head()

In [ ]:
plt.figure(figsize = (10, 6))
plt.title('Target distribution')
plt.xticks(rotation = 30, ha='right')
ax = sns.countplot(x = data['target'])
ax.bar_label(ax.containers[0])
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
len(data['target'].unique())

In [ ]:
classes = data['target'].unique()
classes

In [ ]:
target_dic = {'Streptococcus_pyogenes': 0,
              'Salmonella_enterica' : 1,
              'Enterococcus_hirae' : 2,
              'Escherichia_coli' : 3,
              'Campylobacter_jejuni' : 4,
              'Streptococcus_pneumoniae' : 5,
              'Staphylococcus_aureus' : 6,
              'Escherichia_fergusonii' : 7,
              'Bacteroides_fragilis' : 8,
              'Klebsiella_pneumoniae' : 9}

def convert_into_num(target):
    if target in target_dic.keys():
        return target_dic[target]

In [ ]:
data.head()

# Checking null values

In [ ]:
data.isnull().sum()

# Split Data

In [ ]:
X = data.drop('target', axis=1).to_numpy()
y = data['target']

In [ ]:
X.shape, y.shape

In [ ]:
new_y = [convert_into_num(y_num) for y_num in y]
new_y[:5]

In [ ]:
new_y = np.array(new_y)

# Splitting traning set

In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed(42)

X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, new_y, test_size=0.01, random_state=42)

X_train_valid.shape, X_test.shape, y_train_valid.shape, y_test.shape

# Building and Training our model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier

# Create the parameter grid based on the results of random search 
param_grid = {
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [8, 10, 12],
}
# Create a based model
rf = ExtraTreesClassifier(random_state=42)
# Instantiate the grid search model
model_1 = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 1)

model_1.fit(X_train_valid, y_train_valid)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

plt.figure(figsize=(15, 10))
sns.heatmap(confusion_matrix(y_true=y_test, 
                 y_pred=model_1.predict(X_test)), annot=True,
                 fmt="d");

In [ ]:
model_1.score(X_test, y_test) * 100

# Test Data

In [ ]:
test_dataset = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')

In [ ]:
test_passengerIds = test_dataset['row_id'].values
test_dataset=test_dataset.drop(['row_id'],axis=1)

In [ ]:
test_dataset.head()

In [ ]:
test_dataset.isna().sum()

In [ ]:
test_dataset.info()

In [ ]:
y_pred = model_1.predict(test_dataset)
y_pred.shape

In [ ]:
y_pred[:5]

In [ ]:
target_dic = {0: 'Streptococcus_pyogenes',
              1: 'Salmonella_enterica',
              2: 'Enterococcus_hirae',
              3: 'Escherichia_coli',
              4: 'Campylobacter_jejuni',
              5: 'Streptococcus_pneumoniae',
              6: 'Staphylococcus_aureus',
              7: 'Escherichia_fergusonii',
              8: 'Bacteroides_fragilis',
              9: 'Klebsiella_pneumoniae'}

def convert_into_category(target):
    if target in target_dic.keys():
        return target_dic[target]

In [ ]:
y_pred_new = [convert_into_category(tar) for tar in y_pred]
y_pred_new[:5]

In [ ]:
test_passengerIds.shape

In [ ]:
output = pd.DataFrame({'row_id':test_passengerIds, 'target': y_pred_new})
output.to_csv('submission.csv', index=False)

In [ ]:
output